In [340]:
# pip install yfinance
# pip install python-bcb==0.3.3

In [341]:
# BIBLIOTECAS

import yfinance as yf
from datetime import date
from bcb import sgs
import pandas as pd

In [342]:
#

comeco = "2016-01-01"
final = date.today().strftime("%Y-%m-%d")

# Define a function to load the dataset
def carregar_base(ticker):
    data = yf.download(ticker, comeco, final)
    data = data.rename(columns={'Close': 'Close'})
    data.reset_index(inplace=True)
    return data

In [343]:
df = carregar_base('ABEV3.SA')
df.head()

/tmp/ipython-input-342-3976495122.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, comeco, final)
[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,ABEV3.SA,ABEV3.SA,ABEV3.SA,ABEV3.SA,ABEV3.SA
0,2016-01-04,12.604821,12.985676,12.604821,12.985676,13206900
1,2016-01-05,12.802569,12.831867,12.531578,12.634115,10774200
2,2016-01-06,12.678060,12.802570,12.597495,12.714681,7739100
3,2016-01-07,12.341150,12.685384,12.341150,12.575522,15316400
4,2016-01-08,12.502279,12.597494,12.399742,12.399742,10684000


In [344]:
df.columns # Nome das colunas

MultiIndex([(  'Date',         ''),
            ( 'Close', 'ABEV3.SA'),
            (  'High', 'ABEV3.SA'),
            (   'Low', 'ABEV3.SA'),
            (  'Open', 'ABEV3.SA'),
            ('Volume', 'ABEV3.SA')],
           names=['Price', 'Ticker'])

In [345]:
# Carregar Variáveis Extras

# Dólar
dolar = carregar_base('USDBRL=X')
dolar = dolar[['Date', 'Close']]
dolar.rename(columns={'Close': 'Close_Dolar'}, inplace=True)

# Barril de Petróleo Brent
brent = carregar_base('BZ=F')
brent = brent[['Date', 'Close']]
brent.rename(columns={'Close': 'Close_Petroleo'}, inplace=True)

# Índice da Ibovespa
ibov = carregar_base('^BVSP')
ibov = ibov[['Date', 'Close']]
ibov.rename(columns={'Close': 'Close_Ibovespa'}, inplace=True)

# Taxa Selic
selic = sgs.get(11, start=comeco, end=final)
if selic is None:
    raise Exception("Erro ao obter a taxa Selic. Verifique sua conexão ou o intervalo de datas.")

selic = selic.reset_index()
selic.columns = ['Date', 'Close_Selic']

df = df.merge(dolar, on='Date', how='inner')
df = df.merge(brent, on='Date', how='inner')
df = df.merge(ibov, on='Date', how='inner')
#df = df.merge(selic, on='Date', how='inner')

df.head()

/tmp/ipython-input-342-3976495122.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, comeco, final)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-342-3976495122.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, comeco, final)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-342-3976495122.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, comeco, final)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-345-913177811.py:26: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.merge(dolar, on='Date', how='inner')
/tmp/ipython-input-345-913177811.py:27: PerformanceWarning: dropping on a non-lexsorted multi-index without a 

Price,Date,Close,High,Low,Open,Volume,Close_Dolar,Close_Petroleo,Close_Ibovespa
Ticker,,ABEV3.SA,ABEV3.SA,ABEV3.SA,ABEV3.SA,ABEV3.SA,USDBRL=X,BZ=F,^BVSP
0,2016-01-04,12.604821,12.985676,12.604821,12.985676,13206900,3.9491,37.220001,42141.0
1,2016-01-05,12.802569,12.831867,12.531578,12.634115,10774200,4.0373,36.419998,42419.0
2,2016-01-06,12.678060,12.802570,12.597495,12.714681,7739100,4.0269,34.230000,41773.0
3,2016-01-07,12.341150,12.685384,12.341150,12.575522,15316400,4.0199,33.750000,40695.0
4,2016-01-08,12.502279,12.597494,12.399742,12.399742,10684000,4.0415,33.549999,40612.0


In [346]:
df.columns = [f"{col[0]}_{col[1]}" if col[1] != '' else col[0] for col in df.columns]
df.columns = [col.replace('Close_Dolar_USDBRL=X', 'Close_Dolar').replace('Close_Petroleo_BZ=F', 'Close_Petroleo').replace('Close_Ibovespa_^BVSP', 'Close_Ibovespa') for col in df.columns]
df.columns

Index(['Date', 'Close_ABEV3.SA', 'High_ABEV3.SA', 'Low_ABEV3.SA',
       'Open_ABEV3.SA', 'Volume_ABEV3.SA', 'Close_Dolar', 'Close_Petroleo',
       'Close_Ibovespa'],
      dtype='object')

In [347]:
df.tail()

,Date,Close_ABEV3.SA,High_ABEV3.SA,Low_ABEV3.SA,Open_ABEV3.SA,Volume_ABEV3.SA,Close_Dolar,Close_Petroleo,Close_Ibovespa
2282,2025-06-20,13.53,13.55,13.37,13.41,39451400,5.4900,77.010002,137116.000000
2283,2025-06-23,13.56,13.65,13.47,13.50,27562100,5.5184,71.480003,136551.000000
2284,2025-06-24,13.51,13.65,13.43,13.58,26798300,5.4938,67.139999,137165.000000
2285,2025-06-25,13.15,13.47,13.10,13.40,50901200,5.5103,67.680000,135767.000000
2286,2025-06-26,13.32,0.00,0.00,0.00,0,5.5579,66.860001,137113.890625


In [348]:
df.head(10)

,Date,Close_ABEV3.SA,High_ABEV3.SA,Low_ABEV3.SA,Open_ABEV3.SA,Volume_ABEV3.SA,Close_Dolar,Close_Petroleo,Close_Ibovespa
0,2016-01-04,12.604821,12.985676,12.604821,12.985676,13206900,3.9491,37.220001,42141.0
1,2016-01-05,12.802569,12.831867,12.531578,12.634115,10774200,4.0373,36.419998,42419.0
2,2016-01-06,12.678060,12.802570,12.597495,12.714681,7739100,4.0269,34.230000,41773.0
3,2016-01-07,12.341150,12.685384,12.341150,12.575522,15316400,4.0199,33.750000,40695.0
4,2016-01-08,12.502279,12.597494,12.399742,12.399742,10684000,4.0415,33.549999,40612.0
5,2016-01-11,12.304529,12.663413,12.267909,12.590171,12772700,3.9932,31.549999,39950.0
6,2016-01-12,12.436364,12.480310,12.289883,12.407068,9573800,4.0504,30.860001,39514.0
7,2016-01-13,12.326503,12.634116,12.245937,12.538903,10709300,4.0252,30.309999,38944.0
8,2016-01-14,12.341150,12.407068,12.099454,12.407068,12325600,4.0141,31.030001,39500.0
9,2016-01-15,12.150725,12.231291,11.711278,12.194670,19870300,4.0059,31.010000,38569.0


In [349]:
df.tail(10)

,Date,Close_ABEV3.SA,High_ABEV3.SA,Low_ABEV3.SA,Open_ABEV3.SA,Volume_ABEV3.SA,Close_Dolar,Close_Petroleo,Close_Ibovespa
2277,2025-06-12,13.80,13.89,13.74,13.75,14788400,5.5358,69.360001,137800.000000
2278,2025-06-13,13.64,13.79,13.58,13.68,28248400,5.5373,74.230003,137213.000000
2279,2025-06-16,13.68,13.85,13.59,13.66,27978300,5.5389,73.230003,139256.000000
2280,2025-06-17,13.47,13.75,13.40,13.73,32452200,5.4908,76.449997,138840.000000
2281,2025-06-18,13.49,13.55,13.35,13.38,37567000,5.4930,76.699997,138717.000000
2282,2025-06-20,13.53,13.55,13.37,13.41,39451400,5.4900,77.010002,137116.000000
2283,2025-06-23,13.56,13.65,13.47,13.50,27562100,5.5184,71.480003,136551.000000
2284,2025-06-24,13.51,13.65,13.43,13.58,26798300,5.4938,67.139999,137165.000000
2285,2025-06-25,13.15,13.47,13.10,13.40,50901200,5.5103,67.680000,135767.000000
2286,2025-06-26,13.32,0.00,0.00,0.00,0,5.5579,66.860001,137113.890625


In [350]:
df.shape # Quantidade de linhas e colunas

(2287, 9)

In [351]:
df.info() # Quantidade de dados nulos e tipo dos dados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2287 entries, 0 to 2286
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             2287 non-null   datetime64[ns]
 1   Close_ABEV3.SA   2287 non-null   float64       
 2   High_ABEV3.SA    2287 non-null   float64       
 3   Low_ABEV3.SA     2287 non-null   float64       
 4   Open_ABEV3.SA    2287 non-null   float64       
 5   Volume_ABEV3.SA  2287 non-null   int64         
 6   Close_Dolar      2287 non-null   float64       
 7   Close_Petroleo   2287 non-null   float64       
 8   Close_Ibovespa   2287 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 160.9 KB


In [352]:
df.describe() # Estatísticas descritivas das colunas numéricas

,Date,Close_ABEV3.SA,High_ABEV3.SA,Low_ABEV3.SA,Open_ABEV3.SA,Volume_ABEV3.SA,Close_Dolar,Close_Petroleo,Close_Ibovespa
count,2287,2287.000000,2287.000000,2287.000000,2287.000000,2.287000e+03,2287.000000,2287.000000,2287.000000
mean,2020-10-03 16:36:05.981635328,13.580909,13.735639,13.419930,13.574801,2.318914e+07,4.554494,68.071723,98650.438518
min,2016-01-04 00:00:00,9.123589,0.000000,0.000000,0.000000,0.000000e+00,3.056100,19.990000,37497.000000
25%,2018-05-22 12:00:00,12.616716,12.754173,12.469439,12.606898,1.410710e+07,3.724350,54.680000,78599.000000
50%,2020-10-05 00:00:00,13.534888,13.694435,13.375342,13.526683,2.033570e+07,4.915400,69.019997,104107.000000
75%,2023-02-15 12:00:00,14.449092,14.604705,14.296144,14.456322,2.877440e+07,5.320550,79.699997,118197.500000
max,2025-06-26 00:00:00,19.003433,19.230499,18.815510,19.097389,1.440625e+08,6.300000,127.980003,140110.000000
std,NaN,1.620292,1.637293,1.643628,1.641959,1.346385e+07,0.911756,18.297639,24793.933785
